## Hyperparameter Tuning & Testing Generalizability

In this tutorial, we explore how to tune the hyperparameters of the ggml-ot framework, which is sensitive to choices of $\alpha$ and $\lambda$. These hyperparameters control the margin and the regularization strength in the learned metric.

ggml-ot supports automatic hyperparameter tuning: if a list of possible values for $\alpha$, $\lambda$ or (the rank of the subspace projection) $k$ is provided, instead of a single float, the model will internally search for the best combination.

After training, the most suitable parameters are returned together with the best w_theta.



Load the dataset and import necessary packages. Here, we use the Myocardial Infarction dataset from Kuppe et al., 2022 (https://www.nature.com/articles/s41586-022-05060-x).

In [ ]:
import ggml_ot
import anndata as ad

local_path = "data/czi_dataset.h5ad"
adata = ad.read_h5ad(local_path)

We now run the ggml function and test for the best combination of the values $\alpha$ = {0.1,10} and $\lambda$ = {0.1,10} by passing lists for the two parameters.

To keep the runtime manageable, we limit the number of iterations to four and the subset of cells per patient to 250.
Depending on the size of your dataset and number of hyperparameter combinations, this may take a while.

In [8]:
w_theta, best = ggml_ot.ggml(adata, alpha=[0.1,10], reg=[0.1,10], max_iterations=4, n_splits=3, n_cells=250, verbose=False, n_threads=32)

keeping 7777 high variable genes
Starting the hyperparameter tuning


,knn_acc,mi,ari,vi,score
method,,,,,
"a=0.1, l=0.1, k=5",0.859259,0.814536,0.919751,0.88778,0.866641


,knn_acc,mi,ari,vi,score
method,,,,,
"a=0.1, l=10, k=5",0.688889,0.599336,0.785401,0.754764,0.701028


,knn_acc,mi,ari,vi,score
method,,,,,
"a=10, l=0.1, k=5",0.833333,0.897905,0.959503,0.939445,0.882809


,knn_acc,mi,ari,vi,score
method,,,,,
"a=10, l=10, k=5",0.862963,0.766445,0.889026,0.858675,0.850506


Once tuning is complete, the function returns 
- w_theta: the best learned weight matrix
- best: a dictionary of the best hyperparameter combination that was used to train w_theta

In [9]:
print(best)

{'score': np.float64(0.8828089346826489), 'knn_acc': np.float64(0.8333333333333334), 'alpha': 10, 'reg': 0.1, 'k': 5}


Alternatively, we can run `ggml` and set the parameters `alpha`, `reg` and `rank_k` to [], to start an hyperparameter tuning with default values.

### Evaluating generalizability

If we have a specific hyperparameters to test, we can assess how well they perform on unseen data when w_theta is trained with them. This ensures that the learned metric is not just memorizing training data but captures the general structure in the dataset.

Similar to the hyperparameter tuning, evaluating the generalizability can be done with `ggml`. If an integer value is given for the parameter `n_splits`, w_theta is trained on the given hyperparameters on n_splits different splits. The function evaluates the following measures:

- k-NN classification accuracy
- Adjusted Rand Index (ARI)
- Normalized Mutual Information (NMI)
- Variation of Information (VI)

To evaluate generalizability, we need to define a train/ test split. By specifying the ``n_splits`` parameter, the data is automatically partitioned into n_splits training and test sets, which are then used for evaluation. The parameter also implies the train/ test size - the test size is 1/n_splits and the train size 1 - 1/n_splits. 

In the following example, we create five splits, so an 80% training and 20% test split. The matrix w_theta is trained using the best hyperparameters identified during the tuning step above but with 500 cells per patient and five iterations.

In [4]:
dataset = ggml_ot.scRNA_Dataset(adata, n_cells=500)
w_theta = ggml_ot.ggml(dataset, alpha=10, reg=0.1, max_iterations=5, n_splits=5, n_threads=32)

keeping 7777 high variable genes
Starting the generalizability evaluation
Running GGML with alpha: 10, reg: 0.1, rank: 5


100%|██████████| 19/19 [05:47<00:00, 18.29s/it]


Iteration 1 with Loss  220.24761962890625


100%|██████████| 19/19 [05:45<00:00, 18.19s/it]


Iteration 2 with Loss  143.08209228515625


...

Iteration 4 with Loss  23.992477416992188


100%|██████████| 22/22 [06:31<00:00, 17.78s/it]


Iteration 5 with Loss  21.154428482055664


,knn_acc,mi,ari,vi
method,,,,
"a=10, l=0.1, k=5",0.8,0.895945,0.954769,0.937487


,knn_acc,mi,ari,vi,score
method,,,,,
"a=10, l=0.1, k=5",0.893333,0.917602,0.963777,0.949495,0.918479
